In [243]:
import numpy as np
import pandas as pd
import re
import regex
import nltk
import matplotlib.pyplot as plt
#nltk.download('stopwords')
from nltk.corpus import stopwords

# Storing the training dataset for further use
data = pd.read_csv('train.csv', delimiter = ',')

pd.options.display.max_colwidth = 100

display(data.head(4))

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It By Darrell Luc...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart",Daniel J. Flynn,Ever get the feeling your life circles the roundabout rather than heads in a straight line towar...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, 2016 \nThe tension between intelligence analysts a...",1
3,3,15 Civilians Killed In Single US Airstrike Have Been Identified,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstrike Have Been Identified The rate at which civilia...,1


In [244]:
# Printing the dimensions of the dataset as a reference point for upcoming trimming
print(data.shape)

(20800, 5)


In [245]:
# Removing empty rows
filter = data['text'].str.contains('', regex = True, na = False)

data = data[filter]
print(data.shape)

(20761, 5)


In [246]:
# Creating a new column with the combined data in the title, author and text columns
data['total']=data['title']+' '+data['author']+data['text']

In [247]:
data=data.fillna('')

In [249]:
filter = []

# Removing all rows with non-latin characters
for index, row in data.iterrows():
    reg = regex.match(r'[\p{Latin}\p{posix_punct}]+', row['total'])
    if reg == None:
        filter.append(False)
    else:
        filter.append(True)

In [250]:
filter = np.array(filter)

filter

array([ True,  True,  True, ...,  True,  True,  True])

In [251]:
# Applying filter to the dataset
data = data[filter]

# Printing results of the filter
print("after non-latin removal:", data.shape)

after non-latin removal: (17835, 6)


In [253]:
# Removing newline characters
data['text'].replace(regex=True, inplace=True,to_replace='\n', value='')

# Exporting the dataset to csv format for storage
data.to_csv('newtrain.csv')

In [254]:
# Creating a list with only the texts and none of the metadata
texts = data['text'].tolist()

In [264]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()

# Building a vocabulary from all the texts
vectorizer.fit(texts)
vocabulary = vectorizer.vocabulary_ # Storing vocabulary in new variable for easier use
print("Before stopwords removal:", len(vocabulary)) # Printing the size of the vocabulary prior to removal of stopwords

{'house': 61875, 'dem': 35065, 'aide': 6312, 'we': 140906, 'didn': 36907, 'even': 45432, 'see': 116101, 'comey': 27969, 'letter': 75981, 'until': 136360, 'jason': 68245, 'chaffetz': 24430, 'tweeted': 133869, 'it': 67481, 'by': 21557, 'darrell': 33550, 'lucus': 78247, 'on': 93309, 'october': 92505, '30': 1861, '2016': 1337, 'subscribe': 125194, 'the': 129592, 'stump': 124867, 'in': 64255, 'american': 8053, 'fork': 50169, 'utah': 136944, 'image': 63647, 'courtesy': 31019, 'michael': 83568, 'jolley': 69267, 'available': 12650, 'under': 134990, 'creative': 31423, 'commons': 28175, 'license': 76343, 'with': 142658, 'apologies': 9733, 'to': 130969, 'keith': 71146, 'olbermann': 93036, 'there': 129812, 'is': 67192, 'no': 90509, 'doubt': 39307, 'who': 141912, 'worst': 143246, 'person': 98196, 'world': 143177, 'this': 130039, 'week': 141071, 'fbi': 47645, 'director': 37417, 'james': 68042, 'but': 21419, 'according': 4611, 'democratic': 35195, 'looks': 77741, 'like': 76583, 'also': 7617, 'know': 

In [265]:
# Retrieving a list of English stopwords
stopWords = set(stopwords.words('english'))

# Creating a copy of vocabulary for iteration
temp_dict = dict(vocabulary)
for key in temp_dict:
    if key in stopWords:
        del vocabulary[key] # Removing English stopwords from actual vocabulary

147069


In [267]:
# Printing results of the stopword removal
print("After stopwords removal:", len(vocabulary))

146924


In [276]:
# Exporting the vocabulary to a csv file
pd.DataFrame([vocabulary]).to_csv('Vocabulary.csv', sep=',')